# Travel Agent AI with Plugin

This notebook creates an AI travel agent using OpenAI's `gpt-4o-mini` via GitHub Models. It suggests random vacation destinations and plans trips based on user input.

No `semantic_kernel`—just `openai` and Python. Let’s keep it chill! ✈️

## Import the Needed Python Packages

In [1]:
import os
import random
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import display, HTML

## Create the Client

We’re using [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) to access `gpt-4o-mini`. Make sure your `GITHUB_TOKEN` is set in a `.env` file. You can swap `gpt-4o-mini` for another model to test different results.

In [6]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"),
    base_url="https://models.inference.ai.azure.com/",
)

## Create the Destination Picker

This `DestinationsPlugin` picks random vacation spots, same as the original but without `semantic_kernel`.

In [3]:
class DestinationsPlugin:
    """A list of random vacation destinations."""

    def __init__(self):
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        self.last_destination = None

    def get_random_destination(self) -> str:
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        destination = random.choice(available_destinations)
        self.last_destination = destination
        return destination

## Run the Agent

We’ll process user inputs to plan vacations, using the `DestinationsPlugin` to pick random spots. The output mimics the original’s HTML style, including function call details.

In [4]:
async def run_travel_agent(user_input: str, plugin: DestinationsPlugin) -> tuple[str, str]:
    system_prompt = "You are a helpful AI travel agent that plans vacations for customers at random destinations. Use the provided destination to craft a response."
    destination = plugin.get_random_destination()
    prompt = f"{user_input}\nSuggested destination: {destination}"

    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content, destination

async def main():
    plugin = DestinationsPlugin()
    user_inputs = [
        "Plan me a day trip.",
        "I don't like that destination. Plan me another vacation.",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        # Simulate function call
        response, destination = await run_travel_agent(user_input, plugin)
        function_calls = [f"Calling function: get_random_destination()", f"Function Result:\n\n{destination}"]

        # Display function calls
        html_output += (
            "<div style='margin-bottom:10px'>"
            "<details>"
            "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
            "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
            f"{chr(10).join(function_calls)}"
            "</div></details></div>"
        )

        # Display response
        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>TravelAgent:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{response}</div></div><hr>"
        )

        display(HTML(html_output))

await main()